# Alternative text summarization techniques

Here we experiment with different Python Libraries and techniques to tackle our text summarization task.


In [ ]:
import requests
import json
import time
import os, sys
from IPython.display import display
import datetime
import pandas as pd
import unicodedata
from bs4 import BeautifulSoup
import nltk
import numpy as np
import time

In [ ]:
# Define the path of the directory where the data is stored
data_path = "/home/gkastro/title-prediction-tensorflow/content-data/"

# Loading one of the articles to work on
filename = os.listdir(data_path+"2016/1/")[0]
file = open(data_path+"2016/1/"+str(filename), "r")
content = json.load(file)
file.close()
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
body_raw = unicodedata.normalize("NFKD",content["body"]).encode("ascii", "ignore")
soup = BeautifulSoup(body_raw, "html.parser")
# soup.text
body_text = "\n".join(tokenizer.tokenize(soup.get_text().replace("\n", " ").replace("\r", "").replace("="," ").replace("\t", " ")))
# body_text

### Gensim

In [ ]:
#Gensim will calculate the most important sentences from the document, using TextRank and will return to us a summary
# using only these sentences. It does not generate new sentences,
# it just identifies and copies the most significant sentences.
# More information at https://github.com/RaRe-Technologies/gensim
from gensim.summarization import summarize, keywords

# We can summarize the given article/document by defining the size of the summary 
# as a percentage of the size of the original one
display(summarize(body_text, ratio=0.15))

In [ ]:
# Or as an absolute word count
display(summarize(body_text, word_count=50))

In [ ]:
# We can also extract the keywords from the document
print(keywords(body_text))

### Sumy

In [ ]:
# Sumy provides a variety of algorithms for text summarization, among which also TextRank.
# More information about Sumy can be found at https://pypi.python.org/pypi/sumy

from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

In [ ]:
# Here we get summaries using TextRank, LexRank and LSA.
# All three summaries produced are different, so it might be a good idea to use all three of them to find the 
# sentences on which they are overlapping, these sentences are more likely to contain valuable information.

LANGUAGE = "english"
SENTENCES_COUNT = 1
parser = PlaintextParser.from_string(body_text, Tokenizer(LANGUAGE))
# or for plain text files
# parser = PlaintextParser.from_file("document.txt", Tokenizer(LANGUAGE))
stemmer = Stemmer(LANGUAGE)
summarizers = [LexRankSummarizer, TextRankSummarizer, LsaSummarizer]
pop = []
for summarizer in summarizers:
    if summarizers.index(summarizer) == 0:
        name = "LexRank"
    elif summarizers.index(summarizer) == 1:
        name = "TextRank"
    else:
        name = "LSA"
    print("Summarizing with: "+name)
    summarizer = summarizer(stemmer)
    summarizer.stop_words = get_stop_words(LANGUAGE)
    for sentence in summarizer(parser.document, SENTENCES_COUNT):
        print(sentence)
        # Printing the number of the sentence in the original article
        sen_num = (body_text[0:body_text.index(str(sentence))].count("\n")+1)
        print(sen_num)
        # Inserting the number of the sentence in the popularity list to see which sentences were picked by most models
        # as important
        pop.append(sen_num-1)
    print("\n")

 ### Rake
 Keyword - Keyphrase extraction

In [ ]:
import RAKE
import operator

In [ ]:
rake_object = RAKE.Rake("SmartStoplist.txt")

In [ ]:
keywords = rake_object.run(soup.text)
for key in keywords:
    print(key)
    print("\n")

### Keyword Generator

KG is a command line tool for extracting keywords from text, using it is very simple and doesn't require any code.

We need the documents in .txt format and then run the appropriate command in order to retrieve the keywords.

More info [here](https://github.com/jlonij/keyword-generator)

### De-referencing titles to articles

Our Tensorflow model creates titles and stores them in a txt file, without any information about the article to which the title corresponds.
The following lines of code search in all the articles to find the one corresponding to the title produced, giving back the UUID.

In [ ]:
decode_path = "/home/gkastro/decode/unseen-data/"
for filename in os.listdir(decode_path):
    if filename.startswith("ref"):
        file = open(decode_path+filename, "r")
        decode_data = file.read()
        file.close()

In [ ]:
decode_titles = "".join(decode_data.replace("output=", "")).splitlines()
decode_titles

In [ ]:
data_path = "/home/gkastro/title-prediction-tensorflow/content-data/"

# Loading and searching the article titles
year = 2017
for month in range(1,4):
    for filename in os.listdir(data_path+str(year)+"/"+str(month)):
        file = open(data_path+str(year)+"/"+str(month)+"/"+str(filename), "r")
        content = json.load(file)
        file.close()
        if content["title"]["title"] == decode_titles[0]:
            print(content["apiUrl"][content["apiUrl"].rfind("/")+1:len(content["apiUrl"])])
            break